# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [22]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [24]:
# your code here

#read the dataset and store as a variable
orders_read_csv = pd.read_csv("Orders.zip") 

#create a dataframe from the variable called orders
orders = pd.DataFrame(orders_read_csv)

#display the dataframe head to overview the data
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [25]:
#Drop Unamed column from table as not needed

orders = orders.drop(columns="Unnamed: 0")

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [26]:
#aggregates spending on CustomerID column using groupby method and by the amount spent using sum() method 
aggregated_spending = pd.DataFrame(orders.groupby(["CustomerID"])["amount_spent"].sum())

aggregated_spending

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [27]:
#find the 95th and 7th percentile using quantile() method

percentile_75 = aggregated_spending['amount_spent'].quantile(0.75)

percentile_95 = aggregated_spending['amount_spent'].quantile(0.95)

print(percentile_95, percentile_75)

5840.181999999983 1661.6400000000003


In [28]:
#create an object containing the aggregated spending by amount spent above the 95th percentile
vip_customer = aggregated_spending[aggregated_spending['amount_spent'] >= aggregated_spending['amount_spent'].quantile(0.95)]

#create an object containing the aggregated spending by amount spent above the 75th percentile
preffered_customers = aggregated_spending[aggregated_spending['amount_spent'].between(percentile_75, percentile_95)]

#print to test
vip_customer

#print to test
preffered_customers

,amount_spent
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43
...,...
18259,2338.60
18260,2643.20
18272,3078.58


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [29]:

#create a column that differentiates between Vip and non-Vip customers whether they're in the vip_customer table
orders["vip_customer"] = np.where(orders["CustomerID"].isin(vip_customer.index), "vip", "not_vip")

#print to test
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,vip_customer
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,not_vip
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,not_vip
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip


In [30]:
#create a new table containing only those labelled 'vip'

only_vip = orders[orders["vip_customer"] == "vip"]

#test it worked
only_vip

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,vip_customer
26,536370,22728,2010,12,3,8,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,France,90.0,vip
27,536370,22727,2010,12,3,8,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,France,90.0,vip
28,536370,22726,2010,12,3,8,alarm clock bakelike green,12,2010-12-01 08:45:00,3.75,12583,France,45.0,vip
29,536370,21724,2010,12,3,8,panda and bunnies sticker sheet,12,2010-12-01 08:45:00,0.85,12583,France,10.2,vip
30,536370,21883,2010,12,3,8,stars gift tape,24,2010-12-01 08:45:00,0.65,12583,France,15.6,vip
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397883,581584,85038,2011,12,5,12,6 chocolate love heart t-lights,48,2011-12-09 12:25:00,1.85,13777,United Kingdom,88.8,vip
397905,581586,22061,2011,12,5,12,large cake stand hanging strawbery,8,2011-12-09 12:49:00,2.95,13113,United Kingdom,23.6,vip
397906,581586,23275,2011,12,5,12,set of 3 hanging owls ollie beak,24,2011-12-09 12:49:00,1.25,13113,United Kingdom,30.0,vip
397907,581586,21217,2011,12,5,12,red retrospot round cake tins,24,2011-12-09 12:49:00,8.95,13113,United Kingdom,214.8,vip


In [31]:
#create a table containing relevant columns grouped by the country (as that's what we're interested in)
#count the amount of the times each country appears and sort the list descending to get the highest values for most vip customers which is the UK

only_vip[["vip_customer", "Country"]].groupby(["Country"]).count().sort_values(by="vip_customer", ascending=False)

,vip_customer
Country,
United Kingdom,84185
EIRE,7077
France,3290
Germany,3127
Netherlands,2080
Australia,898
Portugal,681
Switzerland,594
Spain,511


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [32]:
# your code here

#create a column that differentiates between preffered customers whether they're in the preffered_customers table (and the previous Vip/Non-Vip) table

orders["preffered_customer"] = np.where(orders["CustomerID"].isin(preffered_customers.index), "preffered", "non-preffered")

#check it worked 
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,vip_customer,preffered_customer
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,not_vip,preffered
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip,preffered
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,not_vip,preffered
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip,preffered
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip,preffered


In [33]:
#filter customers based on whether they're vip or preffered 

#create a table containing relevant columns grouped by the country (as that's what we're interested in)
#count the amount of the times each country appears and sort the list descending to get the highest values for most vip customers 

vip_preferred = orders[(orders['vip_customer'] == "vip") | (orders['preffered_customer'] == "preffered")]

vip_preferred.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,vip_customer,preffered_customer
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,not_vip,preffered
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip,preffered
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,not_vip,preffered
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip,preffered
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,not_vip,preffered


In [34]:
#create a table containing relevant columns grouped by country and customer id 
#count the amount of the times each country appears for either vip or preffered  and sort the list descending to get the highest values for most vip and preffered customers 

pd.DataFrame(vip_preferred.groupby(['Country'])['CustomerID'].count().sort_values(ascending=False))


#again we can see the UK has the most of both types of customers

,CustomerID
Country,
United Kingdom,221635
Germany,7349
EIRE,7238
France,6301
Netherlands,2080
Spain,1569
Belgium,1557
Switzerland,1370
Portugal,1093
